In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [62]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для CC50.csv')
df

,Chi0n,SlogP_VSA6,MinPartialCharge,BCUT2D_CHGHI,SlogP_VSA1,TPSA,HeavyAtomMolWt,SMR_VSA7,Target feature,Kappa1,...,NHOHCount,EState_VSA3,VSA_EState8,MolLogP,fr_bicyclic,PEOE_VSA6,PEOE_VSA3,SMR_VSA10,EState_VSA2,Target Feature
0,19.534409,9.984809,-0.293526,2.600532,0.000000,24.72,340.300,0.000000,1,20.606247,...,0,21.659962,16.981087,7.1212,4,54.384066,0.000000,11.423370,0.000000,2.244234
1,19.794682,0.000000,-0.313407,2.614066,10.633577,24.06,340.300,0.000000,0,21.163454,...,2,21.659962,17.670565,6.1556,4,54.384066,0.000000,0.000000,0.000000,0.732620
2,23.689110,0.000000,-0.325573,2.665274,0.000000,0.00,388.344,0.000000,0,25.026112,...,0,21.659962,18.287216,7.1292,4,41.542423,0.000000,0.000000,0.000000,2.207210
3,20.241516,9.984809,-0.293526,2.600529,0.000000,24.72,352.311,0.000000,1,21.567454,...,0,21.659962,17.012013,7.5113,4,60.804888,0.000000,11.423370,0.000000,2.032843
4,22.617677,58.515746,-0.257239,2.602486,0.000000,24.72,424.377,59.657840,0,23.194917,...,0,10.829981,14.692318,9.1148,4,65.807891,0.000000,22.798143,10.829981,2.143861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,18.825334,11.649125,-0.468587,2.726237,0.000000,69.67,380.270,11.649125,0,20.263719,...,0,11.835812,8.824371,4.3002,1,38.841158,0.000000,17.907916,52.409521,1.544064
997,21.810933,11.649125,-0.467493,2.725543,5.316789,98.77,446.309,11.649125,0,24.511583,...,1,5.917906,10.378794,3.8049,1,45.764895,4.794537,23.815096,52.347395,1.531471
998,23.633394,11.649125,-0.467485,2.725818,5.316789,98.77,502.398,11.649125,0,27.726151,...,1,12.338728,8.774745,4.5381,1,45.764895,4.794537,35.576981,46.932405,1.531472
999,23.380977,0.000000,-0.468755,2.730109,0.000000,122.27,480.299,0.000000,0,29.111081,...,0,19.262465,7.488627,3.3649,3,27.192033,0.000000,29.660466,30.512100,1.518509


In [63]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [65]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [66]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 124}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.2}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параме

#### Поиск лучшей модели

In [67]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.2675
MAE: 0.4163
R^2: 0.4087

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.2293
MAE: 0.3561
R^2: 0.4931

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.2638
MAE: 0.4124
R^2: 0.4168

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [68]:
best_regression_model.predict(X_test)

Модель для формирования прогноза не сформирована, используйте метод get_best_model


Так как мы не смогли найти модель, которая была бы лучше, чем заданное значение метрикик, изменим значение R^2 score на 0.4

In [69]:
best_regression_model.assessment_criterion['value'] = 0.4

Попробуем снова подобрать модель

In [70]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.2675
MAE: 0.4163
R^2: 0.4087

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.2293
MAE: 0.3561
R^2: 0.4931

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.2638
MAE: 0.4124
R^2: 0.4168

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

In [71]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=4,
                      n_estimators=124, random_state=42)
Предсказание сформировано


array([1.00989748e+03, 4.57558721e+02, 2.19157744e+02, 5.75418054e+01,
       3.58910311e+02, 4.50298410e+02, 8.97083838e+02, 3.50269492e+02,
       5.05656679e+01, 5.71932145e+02, 2.26802492e+01, 1.36341761e+01,
       3.01890953e+02, 6.05292236e+02, 8.58175544e+01, 1.10204418e+02,
       9.24423573e+01, 6.08762574e+00, 1.05109635e+03, 2.76392048e+01,
       1.45627421e+02, 6.47734081e+02, 3.39027417e+02, 8.67318265e+02,
       5.79181141e+02, 2.59953574e+02, 3.17299969e+03, 9.70949444e+01,
       1.07623013e+02, 5.05656679e+01, 2.23792959e+02, 5.82991520e+02,
       8.33878841e+02, 3.99643793e+02, 4.09225841e+02, 1.98155820e+02,
       7.28223435e+02, 3.59135035e+01, 3.96093063e+02, 6.53714776e+02,
       2.70637495e+01, 6.53714776e+02, 4.48888796e+01, 6.81668969e+01,
       1.28685991e+02, 1.03741599e+02, 1.34181680e+02, 4.29374503e+02,
       3.74857733e+02, 4.37628810e+02, 1.37294786e+02, 1.27291153e+01,
       7.92198305e+02, 1.66374823e+02, 8.35764942e+02, 7.81930015e+02,
      